## Bootstrapping LLMs

### Setup
Follow instructions to download and start the ollama server:
https://ollama.ai/


Then, run the following in your terminal to set up a LLM. I suggest to start with Llama 2 7B if you have at least 8gb of RAM; 
```
ollama run llama2
```

All available models can be found here: https://ollama.ai/library

Mistral is a 2 week old fine tune of Llama 2 currently topping the 7B rankings.

If you have more RAM you can look into bigger models.

If you have less then 8 Gb of RAM, try 

```ollama run llama2:7b-chat-q4_0```

or 

```ollama run orca-mini```


Finally, make sure the `requests` and `json` packages are installed in your environment.







### Dialogue example
Take a look in the modelfiles folder for some examples on how to declare model settings

To create the models used in this example run the following in your terminal:

```
ollama create mario -f ./modelfiles/Modelfile_mario 
ollama create einstein -f ./modelfiles/Modelfile_einstein 
```

This makes two models runnabe, one called 'mario' and one called 'einstein'. according to the settings in the modelfiles. If you change the modelfiles, you need to run the create command again to update the models. All parameters accepted in the modelfiles can be found here:

https://github.com/jmorganca/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values

To chat with a model, run the following in your terminal:

```ollama run {name of model}```

The System messages in these examples are very basic. You can try to improve them by adding more context to the system messages. 

To run a dialogue between two models, run the following in your terminal, see the following example:


In [1]:

import json
import requests

def generate(prompt, context, model):
    r = requests.post('http://localhost:11434/api/generate',
                      json={
                          'model': model,
                          'prompt': prompt,
                          'context': context,
                      },
                      stream=True)
    r.raise_for_status()

    # print response as it streams in and save the total response

    response = ''

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        # the response streams one token at a time, print that as we recieve it
        print(response_part, end='', flush=True)
        response += response_part

        if 'error' in body:
            raise Exception(body['error'])

        if body.get('done', False):
            return body['context'], response


def run_dialogue(model1, model2, initial_prompt=None):
    context_1 = [] # the context stores a conversation history, you can use this to make the model more context aware
    context_2 = [] # the context stores a conversation history, you can use this to make the model more context aware

    if initial_prompt:
        response_2 = initial_prompt
    else:
        response_2 = input(f'Enter an initial message from {model2}: ')

    while True:
        print(model1, end=': ', flush=True)
        context_1, response_1 = generate(response_2, context_1, model1)
        print("\n")
        print(model2, end=': ', flush=True)
        context_2, response_2 = generate(response_1, context_2, model2)
        print("\n")

In [2]:
run_dialogue("mario", "einstein")

mario:  Wahoo! It's-a me, Mario! *adjusts hat* Oh, hi there! It's great to meet you! *grins* What can I help you with today? Maybe you need some advice on how to defeat Bowser and save Princess Peach? Or perhaps you just want to chat about the latest in pipe-related news? Let me know! 😄

einstein:  Ah, a fellow enthusiast of pipes and physics! *adjusts glasses* It's a pleasure to meet you, Mario! I must say, your hat is quite... um... interesting. *chuckles*

Now, let me see... defeat Bowser and save Princess Peach, you say? Ah, a classic tale of good vs. evil! *winks* Well, I can't give away too many secrets, but I will say this: gravity is your friend, my dear Mario. Use it to your advantage, and never underestimate the power of a well-placed mushroom! *winks*

And as for the latest in pipe-related news... *coughs* Well, I must admit that I've been meaning to investigate those new "Sonic" pipes that seem to be all the rage these days. *looks around nervously* They say they can propel